Predictions are in yolo format:

0 0.727788 0.340503 0.0609681 0.258929

and should be on label studio format:

[{
	"data": {
	"image": "/data/upload/1/7f1bf9ff-060324-163034-70-temp51.jpg" 
	},
	"predictions": [{
		"model_version": "one",
		"score": 0.5,
		"result": [
		{"id": "result1",
		"type": "rectanglelabels",        
		"from_name": "label", "to_name": "image",
		"original_width": 600, "original_height": 403,
		"image_rotation": 0,
		"value": {
			"rotation": 0,          
			"x": 4.98, "y": 12.82,
			"width": 32.52, "height": 44.91,
			"rectanglelabels": ["Person"]
			}
		},
		{"id": "result2",
		"type": "rectanglelabels",        
		"from_name": "label", "to_name": "image",
		"original_width": 600, "original_height": 403,
		"image_rotation": 0,
		"value": {
			"rotation": 0,          
			"x": 75.47, "y": 82.33,
			"width": 5.74, "height": 7.40,
			"rectanglelabels": ["Person"]
			}
		}]
	}]
}]


In [90]:
import pandas as pd
import cv2
import os

In [91]:
# Path to the raw images, without any bounding boxes of processing
images_source_path = "/home/hallvaeb/data-masteroppgave/FIMUS-3rd-hm12/images"
# Where are the labels that are to be converted to json for label studio import?
labels_source_path = "/home/hallvaeb/data-masteroppgave/FIMUS-3rd-hm12/labels"
# This path is so that label studio can find the images when the predictions are imported. The rest of the path after the database-path opened up for label studio to see.
relative_images_path_for_labels_output				 = "FIMUS-3rd-hm12/images"
# Where to save the json file.
output_json        = '/home/hallvaeb/data-masteroppgave/FIMUS-3rd-hm12/yolo_inferences.json'

In [92]:
def read_yolo_file(file_path, image_id):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    data = []
    for line in lines:
        class_id, x_center, y_center, width, height, conf = line.split()
        data.append([image_id, float(class_id), float(x_center), float(y_center), float(width), float(height), float(conf)])
    df = pd.DataFrame(data, columns=['image_id', 'class_id', 'x_center_norm', 'y_center_norm', 'width_norm', 'height_norm', 'conf'])
    return df

In [93]:
def read_all_yolo_files(labels_source_path):
    all_detections = []
    for file_name in os.listdir(labels_source_path):
        if file_name.endswith('.txt'):
            image_id = os.path.splitext(file_name)[0]
            file_path = os.path.join(labels_source_path, file_name)
            df_detections = read_yolo_file(file_path, image_id)
            all_detections.append(df_detections)
    df_all_detections = pd.concat(all_detections, ignore_index=True)
    return df_all_detections

In [94]:
df_detections = read_all_yolo_files(labels_source_path)

In [95]:
df_detections.head()

,image_id,class_id,x_center_norm,y_center_norm,width_norm,height_norm,conf
0,130524-145947-hm12-awb1.5_1.5_br55_co0_ec0_iso...,0.0,0.475490,0.898133,0.143382,0.198052,0.898047
1,130524-152214-hm12-awb1.5_1.5_br55_co0_ec0_iso...,0.0,0.608456,0.457995,0.036765,0.183036,0.764415
2,130524-153536-hm12-awb1.5_1.5_br55_co0_ec0_iso...,0.0,0.981005,0.416802,0.037378,0.145292,0.644232
3,130524-151700-hm12-awb1.5_1.5_br55_co0_ec0_iso...,0.0,0.931373,0.409294,0.049020,0.134334,0.745881
4,130524-144821-hm12-awb1.5_1.5_br55_co0_ec0_iso...,0.0,0.840533,0.414976,0.065257,0.115666,0.725069


In [96]:
df_detections.describe()

,class_id,x_center_norm,y_center_norm,width_norm,height_norm,conf
count,242.0,242.000000,242.000000,242.000000,242.000000,242.000000
mean,0.0,0.663056,0.498129,0.064569,0.183312,0.804984
std,0.0,0.172193,0.089538,0.025128,0.078465,0.083597
min,0.0,0.235141,0.361201,0.016238,0.073458,0.501905
25%,0.0,0.568091,0.433695,0.047794,0.125914,0.764869
50%,0.0,0.643612,0.465198,0.055913,0.173092,0.830221
75%,0.0,0.827781,0.559152,0.073759,0.226359,0.862698
max,0.0,0.991575,0.898133,0.143382,0.478896,0.926271


In [97]:
# All images have the same resolution
def df_image_shapes_same_resolutions(folder_path):
	df = pd.DataFrame(columns=['image_id', 'image_width', 'image_height'])
	df.image_id = os.listdir(folder_path)
	shape = cv2.imread(os.path.join(folder_path, df.image_id[0])).shape
	df.image_height = shape[0]
	df.image_width = shape[1]
	df.image_id = df.image_id.str.replace('.jpg', '')
	return df

In [98]:
df_image_shapes = df_image_shapes_same_resolutions(images_source_path)

In [99]:
df = pd.merge(df_detections, df_image_shapes, on='image_id', how='inner')

In [100]:
def norm_center_to_denorm_x1y1(df):
	df['x1_denorm'] = (df['x_center_norm'].astype(float) - df['width_norm'].astype(float)/2) * 100
	df['y1_denorm'] = (df['y_center_norm'].astype(float) - df['height_norm'].astype(float)/2) * 100 
	df['w_denormalized'] = df['width_norm'].astype(float) * 100
	df['h_denormalized'] = df['height_norm'].astype(float) * 100

    # Drop now irrelevant columns, and move image shape to last columns
	to_drop= ['class_id', 'x_center_norm','y_center_norm','width_norm','height_norm', 'conf']
	df.drop(to_drop, axis=1, inplace=True)
	
	return df

In [101]:
df_detections_denorm = norm_center_to_denorm_x1y1(df)

In [102]:
df_detections_denorm.describe()

,image_width,image_height,x1_denorm,y1_denorm,w_denormalized,h_denormalized
count,242.0,242.0,242.000000,242.000000,242.000000,242.000000
mean,3264.0,2464.0,63.077172,40.647307,6.456865,18.331244
std,0.0,0.0,17.443208,5.971473,2.512808,7.846542
min,3264.0,2464.0,19.178930,28.571400,1.623770,7.345780
25%,3264.0,2464.0,53.125009,36.688300,4.779410,12.591350
50%,3264.0,2464.0,61.335823,39.955362,5.591300,17.309250
75%,3264.0,2464.0,79.113025,43.597809,7.375920,22.635950
max,3264.0,2464.0,98.345615,79.910700,14.338200,47.889600


In [103]:
def to_json(df):
    grouped = {}
    for d in df.values:
        key = d[0]
        prediction = {
            "model_version": "yolov9_1",
            "result": [{
                "value": {
                    "rotation": 0,
                    "x": round(d[3], 2),
                    "y": round(d[4], 2),
                    "width": round(d[5], 2),
                    "height": round(d[6], 2),
                    "rectanglelabels": ["Person"]
                },
                "id": "result1",
                "from_name": "label",
                "to_name": "image",
                "type": "rectanglelabels",
                "original_width": d[1],
                "original_height": d[2],
                "image_rotation": 0
            }]
        }
        if key not in grouped:
            grouped[key] = {
                "data": {
                    "filename": str(d[0]),
                    "image": f"/data/local-files/?d={relative_images_path_for_labels_output}/{d[0]}.jpg"
                },
                "predictions": []
            }
        grouped[key]["predictions"].append(prediction)

    # Convert the dictionary back to a list
    result = list(grouped.values())
    return result


In [104]:
res = to_json(df_detections_denorm)

In [105]:
res

[{'data': {'filename': '130524-145947-hm12-awb1.5_1.5_br55_co0_ec0_iso640_3264x2464_sm3_ss79989_79989_fr6',
   'image': '/data/local-files/?d=FIMUS-3rd-hm12/images/130524-145947-hm12-awb1.5_1.5_br55_co0_ec0_iso640_3264x2464_sm3_ss79989_79989_fr6.jpg'},
  'predictions': [{'model_version': 'yolov9_1',
    'result': [{'value': {'rotation': 0,
       'x': 40.38,
       'y': 79.91,
       'width': 14.34,
       'height': 19.81,
       'rectanglelabels': ['Person']},
      'id': 'result1',
      'from_name': 'label',
      'to_name': 'image',
      'type': 'rectanglelabels',
      'original_width': 3264,
      'original_height': 2464,
      'image_rotation': 0}]}]},
 {'data': {'filename': '130524-152214-hm12-awb1.5_1.5_br55_co0_ec0_iso640_3264x2464_sm3_ss79989_79989_fr6',
   'image': '/data/local-files/?d=FIMUS-3rd-hm12/images/130524-152214-hm12-awb1.5_1.5_br55_co0_ec0_iso640_3264x2464_sm3_ss79989_79989_fr6.jpg'},
  'predictions': [{'model_version': 'yolov9_1',
    'result': [{'value': {'rot

In [106]:
import json
with open(f'{output_json}', 'w') as f:
    json.dump(res, f)